In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import os
import pandas as pd
import numpy as np

# system contants
input_folder = "input_files"
input_folder_path = os.path.join(os.curdir, input_folder)
asset_prices = {"xrp":"xrp-usd.csv", "doge":"doge-usd.csv", "ada":"ada-usd.csv", "eth":"eth-usd.csv", "btc":"btc-usd.csv", "cad": "cad"}
usd_to_cad_rate = 1.26
#cad_bank_interest_rate = 0.005 _NOT_USED_YET

investment_strategy = {"xrp": .2, "doge": .15, "ada":.15, "eth":.2, "btc": .3, "cad": 0}

roundup_list = {}
# roundup_list["rut_1"] = 1
roundup_list["rut_2"] = 2
# roundup_list["rut_3"] = 3
# roundup_list["rut_4"] = 4
# roundup_list["rut_5"] = 5
# roundup_list["rut_10"] = 10

# Calculate Roundups

In [ ]:
def cibc_data_extractor(df):
    result_df = df.copy()
    result_df.columns = ["date", "merchant", "spent", "paid", "card_number"]
    return result_df[["date", "spent"]]

In [ ]:
def clean_df(df,date_col_name = "date"):
    result = df.dropna()
    result[date_col_name] = np.array(result[date_col_name], dtype=np.datetime64)
    result = result.set_index(date_col_name)
    return result

In [ ]:
def calc_roundups(df):
    
    def calc(row, max_amount):
        spent = row["spent"]
        decimal = spent - int(spent)
        return max_amount - decimal
    
    result = df.copy()
    
    for rut in roundup_list.keys():
        result[rut] = df.apply(lambda row: calc(row, roundup_list[rut]), axis=1)
    return result

In [ ]:
def sort_result(df):
    return df.sort_values(by=["date"], ascending=True)

def roundups_cumsum_col_name(rut_num):
    return rut_num + "_cumsum"

def calc_roundups_cumsum(df):
    for rut in roundup_list.keys():
        col_name = roundups_cumsum_col_name(rut)
        df[col_name] = df[[rut]].cumsum()
    return df
    

In [ ]:
def calc_user_roundups(bank_name, csv_filename):
    bank_report_readers = {"cibc": cibc_data_extractor}
    extractor = bank_report_readers[bank_name]
    statement_file_path = os.path.join(input_folder_path, csv_filename)
    data = pd.read_csv(statement_file_path)
    extracted_bank_data = extractor(data)
    cleaned_bank_data = clean_df(extracted_bank_data)
    result = calc_roundups_cumsum(sort_result(calc_roundups(cleaned_bank_data)))
    return result

def cad_price_data(rows_num, date_col_data, date_col_name):
    cols = ["Open", "High", "Low", "Close"]
    data = np.full((rows_num, 4), 1 / usd_to_cad_rate)
    result = pd.DataFrame(data=data, columns=cols)
    result[date_col_name] = date_col_data
    return result

# Read Crypto Data

In [ ]:
def asset_avg_price_col_name_for(asset_name):
    return asset_name + "_avg_price"

def asset_bought_col_name_for(asset_name, rut_num):
    return asset_name + "_" + rut_num + "_bought"

def asset_cumsum_col_name_for(asset_name, rut_num):
    return asset_name + "_" + rut_num + "_cumsum"

def asset_daily_value_col_name_for(asset_name, rut_num):
    return "total_" + asset_name + "_" + rut_num + "_value"

def portfolio_return_col_name_for(rut_num):
    return "portfolio_retrun_" + rut_num

In [ ]:
def read_asset_prices():
    result = {}
    df_rows_count = 0
    date_col_data = None

    def daily_average_for(df, asset_name):
        df[asset_name + "_avg_price"] = df[["Open", "High", "Low", "Close"]].mean(axis=1)
        df[asset_name + "_avg_price"] = df[asset_name + "_avg_price"] * usd_to_cad_rate
        return df[[asset_avg_price_col_name_for(asset_name), "date"]]

    for asset_name in asset_prices.keys():
        if asset_name == "cad":
            continue
        data = pd.read_csv(os.path.join(input_folder_path, asset_prices[asset_name]))
        data = data.rename(columns={"Date":"date"})
        tmp = daily_average_for(data, asset_name)
        date_col_data = tmp["date"]
        tmp = clean_df(tmp)
        df_rows_count = tmp.shape[0]
        result[asset_name] = tmp
        
    # add CAD asset Dataset
    cad_data = cad_price_data(df_rows_count, date_col_data, "date")
    tmp = daily_average_for(cad_data, "cad")
    tmp = clean_df(tmp)
    result["cad"] = tmp

    return result


# Joining Crypto Data

In [ ]:
def join_roudups_with_crypto(crypto_prices_dict, roundups_df):
    result = roundups_df
    for asset_name in asset_prices.keys():
        result = result.join(crypto_prices_dict[asset_name], on=["date"])
    return result

In [ ]:
def buy_assets(joined_df):
    result = joined_df
    for asset_name in asset_prices.keys():
        avg_price_col = asset_avg_price_col_name_for(asset_name)
        for rut in roundup_list.keys():
            asset_bought_col = asset_bought_col_name_for(asset_name, rut)
            investment_strategy_percentage = investment_strategy[asset_name]
            result[asset_bought_col] = (result[rut] * investment_strategy_percentage) / result[avg_price_col]
    return result

def calc_assets_cumsum(joined_df):
    result = joined_df
    for asset_name in asset_prices.keys():
        for rut in roundup_list.keys():
            asset_bought_col = asset_bought_col_name_for(asset_name, rut)
            asset_cumsum_col = asset_cumsum_col_name_for(asset_name, rut)
            result[asset_cumsum_col] = result[[asset_bought_col]].cumsum()
    return result

def calc_assets_value_cumsum(joined_df):
    result = joined_df
    for asset_name in asset_prices.keys():
        for rut in roundup_list.keys():
            asset_cumsum_col = asset_cumsum_col_name_for(asset_name, rut)
            avg_price_col = asset_avg_price_col_name_for(asset_name)
            asset_value_col = asset_daily_value_col_name_for(asset_name, rut)
            result[asset_value_col] = result[asset_cumsum_col] * result[avg_price_col]
    return result
    
def calc_protfolio_return(df):
    result = df
    for rut in roundup_list.keys():
        portfolio_return_col = portfolio_return_col_name_for(rut)
        tmp = 0
        
        for asset_name in asset_prices.keys():
            asset_value_col = asset_daily_value_col_name_for(asset_name, rut)
            tmp += result[asset_value_col]
            
        result[portfolio_return_col] = tmp
        
    return result

# Main:

In [ ]:
def main(bank_name, statement_csv_filename):
    result = None
    roundups = calc_user_roundups(bank_name, statement_csv_filename)
    roundups.sum()
    
    crypto_prices = read_asset_prices()
    roundups_joined_crypto = join_roudups_with_crypto(crypto_prices, roundups)
    bought_assets = buy_assets(roundups_joined_crypto)
    cumulative_assets = calc_assets_cumsum(bought_assets)
    cumulative_asset_value = calc_assets_value_cumsum(cumulative_assets)
    portfolio_return = calc_protfolio_return(cumulative_asset_value)
    
    result =  cumulative_asset_value
    return result

In [ ]:
investment_result = main("cibc", "amir_cibc.csv")

# Charts

In [ ]:
investment_result.tail(600)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import seaborn as sns
import numpy as np

y_list = {"rut_2_cumsum":"Cumulative Roundup Savings", "portfolio_retrun_rut_2":"Investment Strategy Return"}
plt.gcf().set_size_inches(25, 9)
plt.grid(True)

for y in y_list.keys():
    y_label = y_list[y]
    plt.plot(investment_result.index,
                 investment_result[y], label=y_label)

plt.legend()
plt.ylabel("Canadian Dollar")
plt.show()